# How to use scene and job

This tutorial demonstrates how to create a scene, and fill it from a speos file.
Then this demonstrates how to create a job from the scene, and run it.

## Prerequisites

### Perform imports

In [1]:
from pathlib import Path
import time

from ansys.speos.core import launcher
from ansys.speos.core.kernel.job import ProtoJob
from ansys.speos.core.speos import Speos

# ### Define constants
# Constants help ensure consistency and avoid repetition throughout the example.

HOSTNAME = "localhost"
GRPC_PORT = 50098  # Be sure the Speos GRPC Server has been started on this port.
USE_DOCKER = True  # Set to False if you're running this example locally as a Notebook.

# ### Load assets
# The assets used to run this example are available in the
# [PySpeos repository](https://github.com/ansys/pyspeos/) on GitHub.
#
# > **Note:** Make sure you
# > have downloaded simulation assets and set ``assets_data_path``
# > to point to the assets folder.

if USE_DOCKER:  # Running on the remote server.
    assets_data_path = Path("/app") / "assets"
else:
    assets_data_path = Path("/path/to/your/download/assets/directory")


# ### Start/Connect to Speos RPC Server
# This Python client connects to a server where the Speos engine
# is running as a service. In this example, the server and
# client are the same machine. The launch_local_speos_rpc_method can
# be used to start a local instance of the service.

if USE_DOCKER:
    speos = Speos(host=HOSTNAME, port=GRPC_PORT)
else:
    speos = launcher.launch_local_speos_rpc_server(port=GRPC_PORT)

# ## Scene

# Create an empty scene

In [2]:
my_scene = speos.client.scenes().create()

Load a file to fill the scene

In [3]:
speos_file = str(
    assets_data_path / "LG_50M_Colorimetric_short.sv5" / "LG_50M_Colorimetric_short.sv5"
)
my_scene.load_file(file_uri=speos_file)

Print scene data model

Here it is possible to see that the scene contains two surface sources, one irradiance sensor.

In [4]:
print(my_scene)

ansys.api.speos.scene.v2.Scene
{
    "name": "LG_50M_Colorimetric_short",
    "description": "From /app/assets/LG_50M_Colorimetric_short.sv5/LG_50M_Colorimetric_short.sv5",
    "part_guid": "ec66b9f6-0f49-4e5e-a562-27adbea65496",
    "sources": [
        {
            "name": "Dom Source 2 (0) in SOURCE2",
            "source_guid": "7ab1dbb8-b473-46d1-8673-132814af8b46",
            "surface_properties": {
                "exitance_constant_properties": {
                    "geo_paths": [
                        {
                            "geo_path": "Solid Body in SOURCE2:2920204960/Face in SOURCE2:222",
                            "reverse_normal": false
                        }
                    ]
                }
            },
            "description": "",
            "metadata": {}
        },
        {
            "name": "Surface Source (0) in SOURCE1",
            "source_guid": "97d86dca-fc95-4f99-89f1-3fc7b0da09cb",
            "surface_properties": {
              

## Job

Create a job for the first simulation. When loaded from a speos file, there is always only one
simulation in the scene.

In [5]:
# First create the protobuf message
job_message = ProtoJob(name="my_job")
job_message.scene_guid = my_scene.key  # The job needs a scene guid
job_message.simulation_path = (
    my_scene.get().simulations[0].name
)  # And needs to know which simulation in the scene is involved.
job_message.job_type = ProtoJob.Type.CPU  # Choose type of job, can also be GPU.
job_message.direct_mc_simulation_properties.automatic_save_frequency = 1800
job_message.direct_mc_simulation_properties.stop_condition_rays_number = (
    200000  # Stop condition, here 200000 rays will be sent.
)

# Create the JobLink
job_link = speos.client.jobs().create(message=job_message)

Start the job

In [6]:
job_link.start()

Verify state of the job

In [7]:
job_link.get_state()

state: RUNNING

Wait that the job is finished

In [8]:
job_state_res = job_link.get_state()
while (
    job_state_res.state != ProtoJob.State.FINISHED
    and job_state_res.state != ProtoJob.State.STOPPED
    and job_state_res.state != ProtoJob.State.IN_ERROR
):
    time.sleep(2)

    job_state_res = job_link.get_state()

Retrieve results of the job

Two results are generated : the result of irradiance sensor:
"ASSEMBLY1.DS (0).Dom Irradiance Sensor (0).xmp" and the simulation report in html

In [9]:
results = job_link.get_results().results
print(results)

[upload_response {
  info {
    uri: "931198a3-ea95-4327-b55e-bd187942fb45"
    file_name: "ASSEMBLY1.DS (0).Dom Irradiance Sensor (0).xmp"
    file_size: 1444015
  }
  upload_duration {
    nanos: 2481561
  }
}
, upload_response {
  info {
    uri: "42fc92ee-c067-40d4-88f9-de89dc08cf79"
    file_name: "ASSEMBLY1.DS (0).html"
    file_size: 511279
  }
  upload_duration {
    nanos: 817529
  }
}
]


Once no more needed: delete the job

In [10]:
job_link.delete()

When loading a speos file into a scene, this creates many objects
(source templates, sensor templates, vop template, sop templates).
Then at the end of the example, we just clean all databases

In [11]:
for item in (
    speos.client.scenes().list()
    + speos.client.simulation_templates().list()
    + speos.client.sensor_templates().list()
    + speos.client.source_templates().list()
    + speos.client.intensity_templates().list()
    + speos.client.spectrums().list()
    + speos.client.vop_templates().list()
    + speos.client.sop_templates().list()
    + speos.client.parts().list()
    + speos.client.bodies().list()
    + speos.client.faces().list()
):
    item.delete()

In [12]:
speos.close()

True